# Cache

A common way to optimize applications is by using caching, which involves storing the outputs of certain processes for a set period. For FastAPI, the [`fastapi_cache`](https://github.com/long2ice/fastapi-cache) library provides a convenient way to implement caching, simplifying the process significantly.

## Setup

To run the examples associated with `fastapi_cache`, we need a specific setup: an API and a caching backend. The following Docker Compose file defines both an API service and a Redis service to be used as the caching backend.

In [37]:
%%writefile cache_files/docker-compose.yml
services:
  fast_api:
    image: fastapi_experiment
    container_name: cache_example_api
    volumes:
      - ./app.py:/app.py
    command: /bin/sh -c "\
      pip3 install redis fastapi-cache2[redis] && \
      uvicorn --host 0.0.0.0 --reload app:app"
    network_mode: host
    stdin_open: true
    tty: true
  redis:
    image: redis:7.4.0
    container_name: cache_example_redis
    network_mode: host
    command: redis-server --port 6380

Overwriting cache_files/docker-compose.yml


The following cell starts the application. 

In [39]:
%%bash
cd cache_files/
docker compose up -d &> /dev/null

Don't forget to clean the environment after all.

In [66]:
%%bash
cd cache_files/
docker compose down &> /dev/null

## Check cache

Caching isn't magic; you can find your cached records in the backend you're using.

---

The following example defines a program that returns a random value each time you access the endpoint, requiring you to specify a `user_id`. 

In [40]:
%%writefile cache_files/app.py
from random import random
from collections.abc import AsyncIterator

from fastapi import FastAPI

from fastapi_cache import FastAPICache
from fastapi_cache.decorator import cache
from fastapi_cache.backends.redis import RedisBackend

from redis import asyncio as aioredis

def lifespan(_: FastAPI) -> AsyncIterator[None]:
    redis = aioredis.from_url("redis://localhost:6380")
    FastAPICache.init(RedisBackend(redis), prefix="fastapi-cache")
    yield

app = FastAPI(lifespan=lifespan)

@app.get("/{id}")
@cache(expire=600)
def index(id: int):
    return random()

Overwriting cache_files/app.py


Now you can try making requests to the application.  Different URLs will return different values. 

In [43]:
%%bash
curl -s localhost:8000/10
echo
curl -s localhost:8000/20

0.33599631817687237
0.7828694749719901

However, requests to the same URL will return the same values as before, because the results have been cached from previous calls.

In [44]:
%%bash
curl -s localhost:8000/10
echo
curl -s localhost:8000/20

0.33599631817687237
0.7828694749719901

Let's check if the cached values are present in the Redis database.

In [45]:
!redis-cli -h localhost -p 6380 keys '*'

1) "fastapi-cache::5511deb9b96a93779726aed3f9e4d8bc"
2) "fastapi-cache::c2ab82769e6470b9ef90fa0de4fbb612"


We see two keys that begin with the string "fastapi-cache", just as we specified in the `prefix` argument. The following Python code prints the values of all keys available in the Redis database. 

In [46]:
import redis

r = redis.Redis(host='localhost', port=6380, db=0)

matching_keys = []
cursor = '0'
while cursor != 0:
    cursor, keys = r.scan(cursor=cursor, match='fastapi-cache*')
    matching_keys.extend(keys)

if matching_keys:
    values = r.mget(matching_keys)
    for key, value in zip(matching_keys, values):
        print(f"{key.decode('utf-8')}: {value.decode('utf-8')}")

fastapi-cache::c2ab82769e6470b9ef90fa0de4fbb612: 0.7828694749719901
fastapi-cache::5511deb9b96a93779726aed3f9e4d8bc: 0.33599631817687237


The values retrieved from Redis match the values we received from the API responses.

## Query params

You can also use query parameters with `fastapi_cache`. Each unique combination of query parameters will have its own cached variable.

---

The following API defines an endpoint that uses query parameters.

In [62]:
%%writefile cache_files/app.py
from random import random
from collections.abc import AsyncIterator

from fastapi import FastAPI

from fastapi_cache import FastAPICache
from fastapi_cache.decorator import cache
from fastapi_cache.backends.redis import RedisBackend

from redis import asyncio as aioredis

def lifespan(_: FastAPI) -> AsyncIterator[None]:
    redis = aioredis.from_url("redis://localhost:6380")
    FastAPICache.init(RedisBackend(redis), prefix="fastapi-cache")
    yield

app = FastAPI(lifespan=lifespan)

@app.get("/")
@cache(expire=600)
def index(id: int):
    return random()

Overwriting cache_files/app.py


Now let's try passing different query parameters to the API.

In [65]:
%%bash
curl -s localhost:8000/?id=50
echo
curl -s localhost:8000/?id=60
echo
echo
curl -s localhost:8000/?id=50
echo
curl -s localhost:8000/?id=60

0.6974430279676769
0.36050751146940563

0.6974430279676769
0.36050751146940563

Different query parameters will produce different results. However, identical sets of query parameters will return the same values due to caching.